# TabPFN test

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, RocCurveDisplay
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt

In [2]:
from tabpfn import TabPFNClassifier
from tabpfn.constants import ModelVersion

In [3]:
#kaisaniemi = pd.read_csv("data/Kaisaniemi_2023-2025.csv")
#kumpula = pd.read_csv("data/Kumpula_2025.csv")
malmi  = pd.read_csv("data/Malmi_2023-2025.csv")
#df = kaisaniemi
#df = kumpula
df = malmi
df.shape

(26301, 10)

In [4]:
#df = df[df["Vallitseva sää"].isin(['Ei merkittäviä sääilmiöitä ', 'Heikkoa lumisadetta', 'Heikkoja vesikuuroja', 'Utua', 'Heikkoa vesisadetta'])]
df = df[df["Vallitseva sää"].isin(['Heikkoa lumisadetta', 'Heikkoja vesikuuroja', 'Utua'])]
df.shape

(2629, 10)

In [5]:
X = df.drop("Vallitseva sää", axis=1)
y = df["Vallitseva sää"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

classifier = TabPFNClassifier.create_default_for_version(ModelVersion.V2)
classifier_tabpfn = classifier.fit(X_train, y_train)

RuntimeError: Running on CPU with more than 1000 samples is not allowed by default due to slow performance.
To override this behavior, set the environment variable TABPFN_ALLOW_CPU_LARGE_DATASET=1 or set ignore_pretraining_limits=True.
Alternatively, consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client

In [ ]:
y_predict = classifier_tabpfn.predict(X_test)
y_score = classifier_tabpfn.predict_proba(X_test)

acc = accuracy_score(y_test, y_predict)
precision = precision_score(y_test, y_predict, average=None)
recall = recall_score(y_test, y_predict, average=None)
f1 = f1_score(y_test, y_predict, average=None)
print(f"accuracy: {acc}")
print(f"precision score: {precision}")
print(f"recall score: {recall}")
print(f"f1-score: {f1}")

y_predict = pd.Series(y_predict)
results = pd.concat([y_test.reset_index(drop=True), y_predict.reset_index(drop=True)], axis=1)
results.columns = ["correct_label", "prediction"]
results.head(10)

## One-vs-Rest multiclass ROC

In [ ]:
label_binarizer = LabelBinarizer().fit(y_train)
onehot_y_test = label_binarizer.transform(y_test)
class_of_interest = 'Heikkoa lumisadetta'
class_indices = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]

display = RocCurveDisplay.from_predictions(
    onehot_y_test[:, class_indices],
    y_score[:, class_indices],
    name=f"{class_of_interest} vs the rest",
    curve_kwargs=dict(color="blue"),
    plot_chance_level=True,
    despine=True,
)
_ = display.ax_.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="One-vs-Rest ROC curves: \nHeikkoa lumisadetta vs (Heikkoja vesikuuroja & Utua)",
)

In [ ]:
label_binarizer = LabelBinarizer().fit(y_train)
onehot_y_test = label_binarizer.transform(y_test)
class_of_interest = 'Heikkoja vesikuuroja'
class_indices = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]

display = RocCurveDisplay.from_predictions(
    onehot_y_test[:, class_indices],
    y_score[:, class_indices],
    name=f"{class_of_interest} vs the rest",
    curve_kwargs=dict(color="darkorange"),
    plot_chance_level=True,
    despine=True,
)
_ = display.ax_.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="One-vs-Rest ROC curves: \nHeikkoja vesikuuroja vs (Heikkoa lumisadetta & Utua)",
)

In [ ]:
label_binarizer = LabelBinarizer().fit(y_train)
onehot_y_test = label_binarizer.transform(y_test)
class_of_interest = 'Utua'
class_indices = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]

display = RocCurveDisplay.from_predictions(
    onehot_y_test[:, class_indices],
    y_score[:, class_indices],
    name=f"{class_of_interest} vs the rest",
    curve_kwargs=dict(color="green"),
    plot_chance_level=True,
    despine=True,
)
_ = display.ax_.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="One-vs-Rest ROC curves: \nUtua vs (Heikkoa lumisadetta & Heikkoja vesikuuroja)",
)

In [ ]:
fig, ax = plt.subplots()
colors = ["blue", "darkorange", "green"]
classes = ["Heikkoa lumisadetta", "Heikkoja vesikuuroja", "Utua"]
for class_of_interest, color in zip(classes, colors):
    class_indices = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
    RocCurveDisplay.from_predictions(
        onehot_y_test[:, class_indices],
        y_score[:, class_indices],
        name=f"{class_of_interest} vs the rest",
        curve_kwargs=dict(color=color),
        ax=ax,
        plot_chance_level=(class_indices==2),
        despine=True,
    )
_=ax.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="One-vs-Rest ROC curves:",
)